#### 提升boosting分类器隶属于集成学习模型，它的基本思想是把成百上千个分类准确率较低的树模型组合起来，称为一个准确率很高的模型。
#### 这个模型的特点在于不断迭代，每次迭代生成一颗新的树

In [2]:
#http://blog.csdn.net/xizero00/article/details/73008330
#http://www.picnet.com.au/blogs/guido/post/2016/09/22/xgboost-windows-x64-binaries-for-download/

In [3]:
from xgboost import XGBClassifier

#### 对比随机森林和xgboost在预测泰坦尼克乘员生的预测能力

In [4]:
import pandas as pd
df = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')

In [5]:
df.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [6]:
X = df[['pclass','age','sex']]
Y = df[['survived']]

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [8]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 1 columns):
survived    1313 non-null int64
dtypes: int64(1)
memory usage: 10.3 KB


In [9]:
## 对缺失的age采用均值补齐
X.fillna(X['age'].mean(),inplace=True)

D:\softwares\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [10]:
#分割数据
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test  = train_test_split(X,Y,test_size=0.25,random_state=123)

D:\softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
x_train.head()

,pclass,age,sex
394,2nd,31.194181,male
44,1st,33.000000,male
444,2nd,23.000000,male
511,2nd,18.000000,female
870,3rd,31.194181,male


In [12]:
#需要将类别的信息数字化，如pclass何sex都需要数字化
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(x_train.to_dict(orient='record'))
X_test = vec.transform(x_test.to_dict(orient='record'))

In [13]:
##采用默认的随机森林
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

D:\softwares\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [14]:
print('accuracy of random forest classifier:%s'%(rfc.score(X_test,y_test)))
print('feature_importances:%s'%(list(zip(rfc.feature_importances_,x_train.columns))))

accuracy of random forest classifier:0.793313069909
feature_importances:[(0.35229306095812224, 'pclass'), (0.1098511026823541, 'age'), (0.038684834173080739, 'sex')]


In [15]:
## xgboost分类算法

In [16]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()
xgbc.fit(X_train,y_train)
xgbc.score(X_test,y_test)

D:\softwares\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\softwares\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.80243161094224924

In [63]:
#从运行结果来看,xgboost确实更好的预测潜能

In [ ]:
# 